In [1]:
import torch
import os

In [2]:

os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
os.environ["CUDA_DEVICE_ORDER"]= "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= '2'


In [3]:
file_paths = ['/home/project/new/lost-in-amazon-rainforest/Train/train_data_person_000.xlsx',
              '/home/project/new/lost-in-amazon-rainforest/Train/train_data_person_001.xlsx',
              '/home/project/new/lost-in-amazon-rainforest/Train/train_data_person_002.xlsx',
              '/home/project/new/lost-in-amazon-rainforest/Train/train_data_person_003.xlsx',
              '/home/project/new/lost-in-amazon-rainforest/Train/train_data_person_004.xlsx',
              '/home/project/new/lost-in-amazon-rainforest/Train/train_data_person_005.xlsx',
              '/home/project/new/lost-in-amazon-rainforest/Train/train_data_person_006.xlsx',
              '/home/project/new/lost-in-amazon-rainforest/Train/train_data_person_007.xlsx',
              '/home/project/new/lost-in-amazon-rainforest/Train/train_data_person_008.xlsx',
              '/home/project/new/lost-in-amazon-rainforest/Train/train_data_person_009.xlsx',

            ]

In [4]:
import numpy as np
import pandas as pd

# Initialization of the all_blocks and all_labels arrays as lists
all_blocks = []
all_labels = []
num_rows = 32
block_width = 250  # Width of each block
stride = 50        # Step to move right for the next block
for file_path in file_paths:
    df = pd.read_excel(file_path)
    # Split the dataframe
    datadf = df.iloc[:, :6750]  # All rows, first 6750 columns
    labeldf = df.iloc[:, -1]   # All rows, last column
    print(labeldf.shape)
    # Remove NaN values from labeldf
    labeldf = labeldf.dropna()
    labeldf = labeldf.reset_index(drop=True)
    datadf = datadf.reset_index(drop=True)

    for start_row in range(0, len(datadf), num_rows):
        # Ensure there are enough rows left; if not, break the loop
        if start_row + num_rows > len(datadf):
            break
        # Select the current 32 rows
        current_rows = datadf.iloc[start_row:start_row + num_rows]

        # Iterate across columns with the specified stride
        for start_col in range(0, 6750, stride):
            end_col = start_col + block_width
            if end_col > 6750:
                break  # Ensure we do not exceed column bounds
            
            # Extract the block of data
            block = current_rows.iloc[:, start_col:end_col].values
            all_blocks.append(block)
            
            # Append the corresponding label to all_labels
            all_labels.append(labeldf.iloc[start_row // num_rows])

# Convert lists to NumPy arrays
all_blocks = np.array(all_blocks)
all_labels = np.array(all_labels)
print("Shape of blocks array:", all_blocks.shape)
print("Shape of labels array:", all_labels.shape)
np.savez_compressed('data50.npz', X=all_blocks, Y=all_labels)


(896,)
(896,)
(896,)
(896,)
(896,)
(896,)
(896,)
(896,)
(896,)
(896,)
Shape of blocks array: (36680, 32, 250)
Shape of labels array: (36680,)


In [5]:
from natsort import natsorted
from tqdm import tqdm
from glob import glob
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D

all_blocks = []
all_labels = []

all_files_path = natsorted(glob('merged_data_pkl/*'))

for path in tqdm(all_files_path):
    with open(path, 'rb') as file:
        data_dict = pickle.load(file)
        all_blocks.append(data_dict['data'])
        all_labels.append(data_dict['label'])

X = np.array(all_blocks)
Y = np.array(all_labels)

print(X.shape, Y.shape)


100%|██████████| 8400/8400 [00:00<00:00, 20903.96it/s]


(8400, 32, 250) (8400,)


In [6]:
print(Y)

[0 0 0 ... 8 8 8]


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
